# This notebook is made to test the catalog with paramaters in order to have only one .yaml file to open multiple files 

In [ ]:
import intake
import dask_hpcconfig
from distributed import Client
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf

cluster = dask_hpcconfig.cluster("datarmor-local")
cluster.scale(14)
client = Client(cluster)
client

#### In order to visualize computations, you need to get to the dask extension and fill it this way :
/user/{JUPYTHERHUB_USERNAME}/proxy/{PORT_NUMBER}/status

### Concatenation time is the time needed to concatenate 2 years of data

| paths | Compute | Plot | concat |shape|dims|chunks|concatenation time|
| :---- |:-------:|:----:|:------:|:--:|:---:|:----:|:----------------:|
|marc_f1-mars3d-adour|Yes|Yes|works|Equal|Equal|Equal|4.36 s|
|marc_f1-mars3d-armor|Yes|Yes|works|Different|Different|Different|6.52 s|
|marc_f1-mars3d-finis|Yes|Yes|works|Different|Different|Different|6.78 s|
|marc_f1-mars3d-gironde|Yes|Yes|works|Different|Different|Different|7.8 s|
|marc_f1-mars3d-loire|Yes|Yes| works|Different|Different|Different|7.59 s|
|marc_f1-mars3d-mangae2500-agrif|Yes|Yes|works|Different|Equal|Different|736 ms|
|marc_f1-mars3d-pdc|Yes|Yes|works|Different|Different|Different|5.25 s|
|marc_f1-mars3d-seine|Yes|Yes|works|Different|Different|Different|7.21 s|

In [ ]:
def get_param(catalog, param):
    cat = intake.open_catalog(catalog + ".yaml")
    return cat.metadata["parameters"][param]["allowed"]


def allowed_param(catalog):
    cat = intake.open_catalog(catalog + ".yaml")
    return cat.metadata["allowed_parameters"]

def year_concat(path, catalog, region):
    
    cat = intake.open_catalog(path + catalog + ".yaml")

    concat = [cat[catalog](region=region, year=i).to_dask() 
              for i in get_param(path + catalog, "year")]

    return xr.concat(
        concat, 
        dim="time", 
        coords="minimal", 
        data_vars="minimal", 
        compat="override"
    )

In [ ]:
regions = get_param(
    "/home/datawork-lops-iaocea/catalog/intake/marc_f1-mars3d", "region"
)
regions

In [ ]:
%%time
ds = year_concat(
    "/home/datawork-lops-iaocea/catalog/intake/", "marc_f1-mars3d", regions[7]
)
ds

In [ ]:
ds.unify_chunks().TEMP.isel(level=1).hvplot.quadmesh(
    x="longitude", y="latitude", datashade=True, geo=True
) * gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")